In [1]:
import os
import itertools
import numpy as np
import pandas as pd

In [2]:
import sys
sys.path.append('../src/')

%load_ext autoreload
%autoreload 2
# Importing our custom module(s)
import folds
import utils

In [3]:
'''Code to view category prevalence'''
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
df = pd.read_csv(os.path.join(dataset_path, 'HAM10000_metadata'))
df.dx.value_counts()/df.shape[0]

nv       0.669496
mel      0.111133
bkl      0.109735
bcc      0.051323
akiec    0.032651
vasc     0.014179
df       0.011483
Name: dx, dtype: float64

In [4]:
'''Code to create labels.csv'''
def label_to_category(label, categories):
    if label not in categories: return [0]
    else: return [categories.index(label)+1]

dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
df = pd.read_csv(os.path.join(dataset_path, 'HAM10000_metadata'))
categories = ['bkl', 'nv', 'df', 'mel', 'vasc', 'bcc', 'akiec']
categories = ['bkl', 'nv', 'mel'] # categories based off prevalence
df['label'] = df['dx'].apply(lambda label: label_to_category(label, categories))
df['path'] = df['image_id'].apply(lambda image_id: os.path.join(dataset_path, 'train/{}.jpg'.format(image_id)))
temp_df = df[['lesion_id', 'label', 'path']].set_index('lesion_id')
temp_df.to_csv(os.path.join(dataset_path, 'train/labels.csv'))
temp_df.head()

,label,path
lesion_id,,
HAM_0000118,[1],/cluster/tufts/hugheslab/eharve06/HAM10000/tra...
HAM_0000118,[1],/cluster/tufts/hugheslab/eharve06/HAM10000/tra...
HAM_0002730,[1],/cluster/tufts/hugheslab/eharve06/HAM10000/tra...
HAM_0002730,[1],/cluster/tufts/hugheslab/eharve06/HAM10000/tra...
HAM_0001466,[1],/cluster/tufts/hugheslab/eharve06/HAM10000/tra...


In [5]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
# TODO: Remove ISIC_0035068.jpg from ISIC2018_Task3_Test_GroundTruth.csv,
#       the image is missing from ISIC2018_Task3_Test_Images.zip
df = pd.read_csv(os.path.join(dataset_path, 'ISIC2018_Task3_Test_GroundTruth.csv'))
categories = ['bkl', 'nv', 'df', 'mel', 'vasc', 'bcc', 'akiec']
categories = ['bkl', 'nv', 'mel'] # categories based off prevalence
df['label'] = df['dx'].apply(lambda label: label_to_category(label, categories))
df['path'] = df['image_id'].apply(lambda image_id: os.path.join(dataset_path, 'test/{}.jpg'.format(image_id)))
temp_df = df[['lesion_id', 'label', 'path']].set_index('lesion_id')
temp_df.to_csv(os.path.join(dataset_path, 'test/labels.csv'))
temp_df.head()

,label,path
lesion_id,,
HAMTEST_0000000,[2],/cluster/tufts/hugheslab/eharve06/HAM10000/tes...
HAMTEST_0000001,[2],/cluster/tufts/hugheslab/eharve06/HAM10000/tes...
HAMTEST_0000002,[1],/cluster/tufts/hugheslab/eharve06/HAM10000/tes...
HAMTEST_0000003,[2],/cluster/tufts/hugheslab/eharve06/HAM10000/tes...
HAMTEST_0000004,[2],/cluster/tufts/hugheslab/eharve06/HAM10000/tes...


In [6]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
n = 100
tune = False
random_state = 3001
augmented_train_dataset, train_dataset, val_or_test_dataset = utils.get_ham10000_datasets(dataset_path, n, tune=tune, random_state=random_state)
print(len(augmented_train_dataset))
print(len(train_dataset))
print(len(val_or_test_dataset))

100
100
1511


In [7]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000'
lr_0s = np.logspace(-1, -4, num=4)
ns = [100, 1000]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
random_states = [1001, 2001, 3001]
prior_type = 'nonlearned'
wandb_project = 'tuned_HAM10000'
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
count = -1
for lr_0, n, random_state, weight_decay in itertools.product(lr_0s, ns, random_states, weight_decays):
    count += 1
    model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
    .format(prior_type, lr_0, n, random_state, weight_decay)
    print('    "python ../src/HAM10000_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --prior_type=\'{}\' --random_state={} --tune --wandb --wandb_project=\'{}\' --weight_decay={}"'\
          .format(dataset_path, experiments_path, lr_0, model_name, n, prior_path, prior_type, random_state, wandb_project, weight_decay))
print(count)

    "python ../src/HAM10000_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/HAM10000' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000' --lr_0=0.1 --model_name='nonlearned_lr_0=0.1_n=100_random_state=1001_weight_decay=0.01' --n=100 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='nonlearned' --random_state=1001 --tune --wandb --wandb_project='tuned_HAM10000' --weight_decay=0.01"
    "python ../src/HAM10000_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/HAM10000' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000' --lr_0=0.1 --model_name='nonlearned_lr_0=0.1_n=100_random_state=1001_weight_decay=0.001' --n=100 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='nonlearned' --random_state=1001 --tune --wandb --wandb_project='tuned_HAM10000' --weight_decay=0.001"
    "python ../src/HAM10000_main.py --da

In [8]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000'
lr_0s = np.logspace(-1, -4, num=4)
ns = [100, 1000]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
prior_type = 'adapted'
random_states = [1001, 2001, 3001]
wandb_project = 'tuned_HAM10000'
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
count = -1
for lr_0, n, random_state, weight_decay in itertools.product(lr_0s, ns, random_states, weight_decays):
    count += 1
    model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
    .format(prior_type, lr_0, n, random_state, weight_decay)
    print('    "python ../src/HAM10000_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --prior_type=\'{}\' --random_state={} --tune --wandb --wandb_project=\'{}\' --weight_decay={}"'\
          .format(dataset_path, experiments_path, lr_0, model_name, n, prior_path, prior_type, random_state, wandb_project, weight_decay))
print(count)

    "python ../src/HAM10000_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/HAM10000' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000' --lr_0=0.1 --model_name='adapted_lr_0=0.1_n=100_random_state=1001_weight_decay=0.01' --n=100 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='adapted' --random_state=1001 --tune --wandb --wandb_project='tuned_HAM10000' --weight_decay=0.01"
    "python ../src/HAM10000_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/HAM10000' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000' --lr_0=0.1 --model_name='adapted_lr_0=0.1_n=100_random_state=1001_weight_decay=0.001' --n=100 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='adapted' --random_state=1001 --tune --wandb --wandb_project='tuned_HAM10000' --weight_decay=0.001"
    "python ../src/HAM10000_main.py --dataset_path='

In [10]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000'
lr_0s = np.logspace(-1, -4, num=4)
ns = [1000]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
prior_scales = np.logspace(0, 9, num=10)
prior_type = 'learned'
random_states = [1001, 2001, 3001]
wandb_project = 'tuned_HAM10000'
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
count = -1
for lr_0, n, prior_scale, random_state, weight_decay in itertools.product(lr_0s, ns, prior_scales, random_states, weight_decays):
    model_name = '{}_lr_0={}_n={}_prior_scale={}_random_state={}_weight_decay={}'\
    .format(prior_type, lr_0, n, prior_scale, random_state, weight_decay)
    count += 1
    print('    "python ../src/HAM10000_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --prior_scale={} --prior_type=\'{}\' --random_state={} --tune --wandb --wandb_project=\'{}\' --weight_decay={}"'\
          .format(dataset_path, experiments_path, lr_0, model_name, n, prior_path, prior_scale, prior_type, random_state, wandb_project, weight_decay))
print(count)

    "python ../src/HAM10000_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/HAM10000' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000' --lr_0=0.1 --model_name='learned_lr_0=0.1_n=1000_prior_scale=100.0_random_state=1001_weight_decay=1e-06' --n=1000 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_scale=100.0 --prior_type='learned' --random_state=1001 --tune --wandb --wandb_project='tuned_HAM10000' --weight_decay=1e-06"
    "python ../src/HAM10000_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/HAM10000' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000' --lr_0=0.1 --model_name='learned_lr_0=0.1_n=1000_prior_scale=100.0_random_state=1001_weight_decay=0.0' --n=1000 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_scale=100.0 --prior_type='learned' --random_state=1001 --tune --wandb --wandb_project='tuned_HAM1

In [10]:
def get_df(path):
    df = pd.read_csv(path, index_col='Unnamed: 0')
    return df

def get_val_loss(df):
    return df.val_or_test_loss.values[-1]

def get_val_acc(df):
    return df.val_or_test_acc.values[-1]

def get_last_epoch(df):
    return df.iloc[-1]

def get_hyperparameters(experiments_path, lr_0s, ns, prior_scales, prior_type, random_states, weight_decays):
    columns = ['lr_0', 'n', 'prior_scale', 'prior_type', 'random_state', 'val_acc', 'weight_decay']
    df = pd.DataFrame(columns=columns)
    for n, prior_scale, random_state in itertools.product(ns, prior_scales, random_states):
        best_val_loss = np.inf
        best_hyperparameters = None
        for lr_0, weight_decay in itertools.product(lr_0s, weight_decays):
            if prior_scale:
                model_name = '{}_lr_0={}_n={}_prior_scale={}_random_state={}_weight_decay={}'\
                .format(prior_type, lr_0, n, prior_scale, random_state, weight_decay)
            else:
                model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
                .format(prior_type, lr_0, n, random_state, weight_decay)
            path =  '{}/{}.csv'.format(experiments_path, model_name)
            val_loss = get_val_loss(get_df(path))
            val_acc = get_val_acc(get_df(path))
            if val_loss < best_val_loss: best_val_loss = val_loss; best_hyperparameters = [lr_0, n, prior_scale, prior_type, random_state, val_acc, weight_decay]
        df.loc[df.shape[0]] = best_hyperparameters
    return df

def get_best_hyperparameters(experiments_path, lr_0s, ns, prior_scales, prior_type, random_states, weight_decays):
    columns = ['lr_0', 'n', 'prior_scale', 'prior_type', 'random_state', 'val_acc', 'weight_decay']
    df = pd.DataFrame(columns=columns)
    for n, random_state in itertools.product(ns, random_states):
        best_val_loss = np.inf
        best_hyperparameters = None
        for lr_0, prior_scale, weight_decay in itertools.product(lr_0s, prior_scales, weight_decays):
            if prior_scale:
                model_name = '{}_lr_0={}_n={}_prior_scale={}_random_state={}_weight_decay={}'\
                .format(prior_type, lr_0, n, prior_scale, random_state, weight_decay)
            else:
                model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
                .format(prior_type, lr_0, n, random_state, weight_decay)
            path =  '{}/{}.csv'.format(experiments_path, model_name)
            val_loss = get_val_loss(get_df(path))
            val_acc = get_val_acc(get_df(path))
            if val_loss < best_val_loss: best_val_loss = val_loss; best_hyperparameters = [lr_0, n, prior_scale, prior_type, random_state, val_acc, weight_decay]
        df.loc[df.shape[0]] = best_hyperparameters
    return df

def get_results(df, experiments_path):
    columns = ['n', 'prior_scale', 'prior_type', 'random_state', 'test_acc', 'test_loss', 'test_nll', 
           'test_prior', 'train_acc', 'train_loss', 'train_nll', 'train_prior']
    results = pd.DataFrame(columns=columns)
    for row_index, row in df.iterrows():
        if row.prior_scale:
            model_name = '{}_lr_0={}_n={}_prior_scale={}_random_state={}_weight_decay={}'\
            .format(row.prior_type, row.lr_0, row.n, row.prior_scale, row.random_state, row.weight_decay)
        else:
            model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
            .format(row.prior_type, row.lr_0, row.n, row.random_state, row.weight_decay)
        path =  '{}/{}.csv'.format(experiments_path, model_name)
        last_epoch = get_last_epoch(get_df(path))
        results_row = [int(row.n), row.prior_scale, row.prior_type, int(row.random_state), 
                       last_epoch.val_or_test_acc, last_epoch.val_or_test_loss, 
                       last_epoch.val_or_test_nll, last_epoch.val_or_test_prior, 
                       last_epoch.train_acc, last_epoch.train_loss, 
                       last_epoch.train_nll, last_epoch.train_prior]
        results.loc[results.shape[0]] = results_row
    return results

In [11]:
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000'
lr_0s = np.logspace(-1, -4, num=4)
ns = [100, 1000]
prior_scales = [None]
prior_type = 'nonlearned'
random_states = [1001, 2001, 3001]
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
nonlearned_hyperparameters = get_best_hyperparameters(experiments_path, lr_0s, ns, prior_scales, prior_type, random_states, weight_decays)
nonlearned_hyperparameters

,lr_0,n,prior_scale,prior_type,random_state,val_acc,weight_decay
0,0.0001,100,None,nonlearned,1001,0.610491,0.010000
1,0.0001,100,None,nonlearned,2001,0.383929,0.000010
2,0.0001,100,None,nonlearned,3001,0.820238,0.000001
3,0.0001,1000,None,nonlearned,1001,0.858887,0.010000
4,0.0001,1000,None,nonlearned,2001,0.885370,0.010000
5,0.0100,1000,None,nonlearned,3001,0.876476,0.010000


In [12]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_HAM10000'
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
wandb_project = 'retrained_HAM10000'
count = -1
for row_index, row in nonlearned_hyperparameters.iterrows():
    if row.prior_scale:
        model_name = '{}_lr_0={}_n={}_prior_scale={}_random_state={}_weight_decay={}'\
        .format(row.prior_type, row.lr_0, row.n, row.prior_scale, row.random_state, row.weight_decay)
        count += 1
        print('    "python ../src/HAM10000_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --prior_scale={} --prior_type=\'{}\' --random_state={} --wandb --wandb_project=\'{}\' --weight_decay={}"'\
              .format(dataset_path, experiments_path, row.lr_0, model_name, int(row.n), prior_path, row.prior_scale, row.prior_type, int(row.random_state), wandb_project, row.weight_decay))
    else:
        model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
        .format(row.prior_type, row.lr_0, row.n, row.random_state, row.weight_decay)
        count += 1
        print('    "python ../src/HAM10000_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --prior_type=\'{}\' --random_state={} --wandb --wandb_project=\'{}\' --weight_decay={}"'\
              .format(dataset_path, experiments_path, row.lr_0, model_name, int(row.n), prior_path, row.prior_type, int(row.random_state), wandb_project, row.weight_decay))
print(count)

    "python ../src/HAM10000_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/HAM10000' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_HAM10000' --lr_0=0.0001 --model_name='nonlearned_lr_0=0.0001_n=100_random_state=1001_weight_decay=0.01' --n=100 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='nonlearned' --random_state=1001 --wandb --wandb_project='retrained_HAM10000' --weight_decay=0.01"
    "python ../src/HAM10000_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/HAM10000' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_HAM10000' --lr_0=0.0001 --model_name='nonlearned_lr_0=0.0001_n=100_random_state=2001_weight_decay=1e-05' --n=100 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='nonlearned' --random_state=2001 --wandb --wandb_project='retrained_HAM10000' --weight_decay=1e-05"
    "python ../src/HAM1000

In [13]:
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000'
lr_0s = np.logspace(-1, -4, num=4)
ns = [100, 1000]
prior_scales = [None]
prior_type = 'adapted'
random_states = [1001, 2001, 3001]
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
adapted_hyperparameters = get_best_hyperparameters(experiments_path, lr_0s, ns, prior_scales, prior_type, random_states, weight_decays)
adapted_hyperparameters

,lr_0,n,prior_scale,prior_type,random_state,val_acc,weight_decay
0,0.0001,100,None,adapted,1001,0.610491,0.000010
1,0.0001,100,None,adapted,2001,0.383929,0.010000
2,0.0001,100,None,adapted,3001,0.820238,0.001000
3,0.0001,1000,None,adapted,1001,0.858799,0.000001
4,0.0001,1000,None,adapted,2001,0.885645,0.000000
5,0.0001,1000,None,adapted,3001,0.854579,0.000010


In [14]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_HAM10000'
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
wandb_project = 'retrained_HAM10000'
count = -1
for row_index, row in adapted_hyperparameters.iterrows():
    if row.prior_scale:
        model_name = '{}_lr_0={}_n={}_prior_scale={}_random_state={}_weight_decay={}'\
        .format(row.prior_type, row.lr_0, row.n, row.prior_scale, row.random_state, row.weight_decay)
        count += 1
        print('    "python ../src/HAM10000_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --prior_scale={} --prior_type=\'{}\' --random_state={} --wandb --wandb_project=\'{}\' --weight_decay={}"'\
              .format(dataset_path, experiments_path, row.lr_0, model_name, int(row.n), prior_path, row.prior_scale, row.prior_type, int(row.random_state), wandb_project, row.weight_decay))
    else:
        model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
        .format(row.prior_type, row.lr_0, row.n, row.random_state, row.weight_decay)
        count += 1
        print('    "python ../src/HAM10000_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --prior_type=\'{}\' --random_state={} --wandb --wandb_project=\'{}\' --weight_decay={}"'\
              .format(dataset_path, experiments_path, row.lr_0, model_name, int(row.n), prior_path, row.prior_type, int(row.random_state), wandb_project, row.weight_decay))
print(count)

    "python ../src/HAM10000_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/HAM10000' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_HAM10000' --lr_0=0.0001 --model_name='adapted_lr_0=0.0001_n=100_random_state=1001_weight_decay=1e-05' --n=100 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='adapted' --random_state=1001 --wandb --wandb_project='retrained_HAM10000' --weight_decay=1e-05"
    "python ../src/HAM10000_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/HAM10000' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_HAM10000' --lr_0=0.0001 --model_name='adapted_lr_0=0.0001_n=100_random_state=2001_weight_decay=0.01' --n=100 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='adapted' --random_state=2001 --wandb --wandb_project='retrained_HAM10000' --weight_decay=0.01"
    "python ../src/HAM10000_main.py --

In [15]:
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_HAM10000'
nonlearned_results = get_results(nonlearned_hyperparameters, experiments_path)
adapted_results = get_results(adapted_hyperparameters, experiments_path)
results = pd.concat([nonlearned_results, adapted_results])
results

,n,prior_scale,prior_type,random_state,test_acc,test_loss,test_nll,test_prior,train_acc,train_loss,train_nll,train_prior
0,100,None,nonlearned,1001,0.750381,1.242558,1.242558,0.000000e+00,1.000000,0.008067,0.008067,0.000000e+00
1,100,None,nonlearned,2001,0.763326,1.328555,1.328555,0.000000e+00,1.000000,0.006217,0.006217,0.000000e+00
2,100,None,nonlearned,3001,0.827820,0.914008,0.914008,0.000000e+00,1.000000,0.007999,0.007999,0.000000e+00
3,1000,None,nonlearned,1001,0.855547,0.789375,0.789375,0.000000e+00,0.984201,0.255172,0.255172,0.000000e+00
4,1000,None,nonlearned,2001,0.863362,0.918982,0.918982,0.000000e+00,0.999973,0.032053,0.032053,0.000000e+00
5,1000,None,nonlearned,3001,0.849623,0.997314,0.997314,0.000000e+00,1.000000,0.008948,0.008948,0.000000e+00
0,100,None,adapted,1001,0.749754,1.261682,1.261675,7.844278e-06,1.000000,0.007479,0.007471,7.844278e-06
1,100,None,adapted,2001,0.762752,1.329374,1.322689,6.684931e-03,1.000000,0.013367,0.006682,6.684931e-03
2,100,None,adapted,3001,0.827833,0.914363,0.913321,1.041261e-03,1.000000,0.009121,0.008079,1.041261e-03
3,1000,None,adapted,1001,0.855534,0.791912,0.791912,8.756663e-07,0.984551,0.250766,0.250766,8.756663e-07


In [18]:
grouped_results = results.groupby(['n', 'prior_type']).agg(lambda x: tuple(x))
columns = ['test_acc', 'train_acc']
for column in columns:
    grouped_results['{}_mean'.format(column)] = grouped_results[column].apply(lambda item: np.mean(item))
    grouped_results['{}_std'.format(column)] = grouped_results[column].apply(lambda item: np.std(item))
grouped_results = grouped_results.reset_index()
grouped_results

,n,prior_type,prior_scale,random_state,test_acc,test_loss,test_nll,test_prior,train_acc,train_loss,train_nll,train_prior,test_acc_mean,test_acc_std,train_acc_mean,train_acc_std
0,100,adapted,"(None, None, None)","(1001, 2001, 3001)","(0.7497538328170776, 0.7627518177032471, 0.827...","(1.2616823982671743, 1.329373971505484, 0.9143...","(1.2616745304540635, 1.3226890721753448, 0.913...","(7.844278115953786e-06, 0.0066849314607679, 0....","(1.0, 1.0, 1.0)","(0.0074785207398235, 0.0133672561496496, 0.009...","(0.0074706766754388, 0.0066823242232203, 0.008...","(7.844278115953784e-06, 0.0066849314607679, 0....",0.780113,0.034158,1.000000,0.000000
1,100,nonlearned,"(None, None, None)","(1001, 2001, 3001)","(0.7503811120986938, 0.7633255124092102, 0.827...","(1.2425584793879765, 1.328555064908248, 0.9140...","(1.2425584793879765, 1.328555064908248, 0.9140...","(0.0, 0.0, 0.0)","(1.0, 1.0, 1.0)","(0.0080669019371271, 0.0062173777259886, 0.007...","(0.0080669019371271, 0.0062173777259886, 0.007...","(0.0, 0.0, 0.0)",0.780509,0.033869,1.000000,0.000000
2,1000,adapted,"(None, None, None)","(1001, 2001, 3001)","(0.8555335998535156, 0.8642812967300415, 0.874...","(0.7919124439802491, 0.9211355100634236, 0.793...","(0.7919115716459576, 0.9211355100634236, 0.793...","(8.756663305575784e-07, 0.0, 1.069778409146238...","(0.9845512509346008, 0.999961256980896, 0.9981...","(0.2507664711475372, 0.0329052025675773, 0.095...","(0.2507655977010726, 0.0329052025675773, 0.095...","(8.756663305575785e-07, 0.0, 1.069778409146238...",0.864798,0.007783,0.994235,0.006886
3,1000,nonlearned,"(None, None, None)","(1001, 2001, 3001)","(0.8555468320846558, 0.863362193107605, 0.8496...","(0.7893752272282115, 0.918981594648052, 0.9973...","(0.7893752272282115, 0.918981594648052, 0.9973...","(0.0, 0.0, 0.0)","(0.9842013120651244, 0.9999729990959167, 1.0)","(0.2551718409061432, 0.0320530468821525, 0.008...","(0.2551718409061432, 0.0320530468821525, 0.008...","(0.0, 0.0, 0.0)",0.856177,0.005627,0.994725,0.007441


In [19]:
grouped_results[['n', 'prior_type', 'test_acc_mean', 'test_acc_std']]

,n,prior_type,test_acc_mean,test_acc_std
0,100,adapted,0.780113,0.034158
1,100,nonlearned,0.780509,0.033869
2,1000,adapted,0.864798,0.007783
3,1000,nonlearned,0.856177,0.005627
